# IDA 2016 Missing Value Handling

In [ ]:
import sys
sys.path.append("..")
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql.session import SparkSession
from helpers.path_translation import translate_to_file_string
from helpers.data_prep_and_print import print_df

In [ ]:
input_file = translate_to_file_string("../data/ida_2016_training_set_update_with_mv.csv")

In [ ]:
spark = (SparkSession
       .builder   
       .master("local[*]")
       .appName("Ida2016 Missing Value Handling")
       .getOrCreate())

## Read the input file

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ",") \
       .csv(input_file) 
print(df.count())
df.printSchema()


# Remove Attributes with >= 60% Missing Values

In [ ]:
# Calculate the number of Missing Values in each column
dict_null_col = {col:df.filter(df[col].isNull()).count() for col in df.columns}
num_rows = df.count()
cols_to_be_removed = [key for key in dict_null_col if dict_null_col[key] >= 0.6*num_rows]
print(cols_to_be_removed)
# Remove columns with more than 60% missing values
df_wnc= df.drop(*cols_to_be_removed)

## Fill Missing Values in Remaining Columns

### Define the columns with Missing Values

In [ ]:
cols_with_null_values = [key for key in dict_null_col if dict_null_col[key] > 0]
cols_with_null_values_to_fill = list(set(cols_with_null_values) - set(cols_to_be_removed))
print(len(cols_with_null_values))
print(len(cols_with_null_values_to_fill))
print(cols_with_null_values_to_fill)

### Calculate the Mean Values of these Columns

In [ ]:

dict_avg_col = {col:df_wnc.agg({col: 'mean'}).collect()[0][0] for col in cols_with_null_values_to_fill}             
print (dict_avg_col)

### Fill the columns with the corresponding mean values

In [ ]:
result_df = df_wnc.fillna(dict_avg_col)
print_df(result_df,5)

## Write the result to csv file

In [ ]:
output_file = "../data/ida_2016_filled_mv.csv"
result_df.write.option("header", "true").csv(output_file)
# result_df.toPandas().to_csv(output_file,index=False) # Save the data into a single file to a permanent location